# 6. Convolutional Neural Networks

### 6.1 From Dense Layers to Convolutions

The models that were discussed in previous chapters are fine options if yopu are dealing with tabular data. By tabular we mean that the data consists of rows corresponding to examples and columns corresponding to features. With tabular data, we might anticipate that pattern we seek could require modeling interactions among the features, but do not assume anything a priori about which features are related to each other or in what way.

Sometimes we truly may not have any knowledge to guide the construction of more cleverly-organized architectures. In these cases, a multilayer perceptron is often the best that we can do. However, once we start dealing with high-dimensional perceptual data, these *structure-less* networks can grow unwieldy.

For instance, let's return to our running example of distinguishing cats from dogs. Say that we do a thorough job in data collection, collecting an annotated sets of high-quality 1-megapixel photographs. This means that the input into a network has *1 million dimensions* (1-megapixel is 1 million pixels). Even an aggresive reduction to *1,000 hidden dimensions* would require a *dense* (fully-connected) layer to support $10^9$ parameters. Unless we have an extremely large dataset (perhaps billions of photos), lots of GPUs, a talent for extreme distributed optimization, and an extraordinary amount of patience, learning the parameters of this network may turn out to be impossible. 

A careful reader might object to this argument on the basis that 1 megapixel resolution may not be necessary. However, while you could get away with 100,000 pixels, we grossly underestimated the number of hidden nodes that it typically takes to learn good hidden representations of images. Learning a binary classifier with so many parameters might seem to require that we collect an enormous dataset, perhaps comparable to the number of dogs and cats on the planet. And yet both humans and computers are able to distinguish cats from dogs quite well, seemingly contradicting these conclusions. That is because images exhibit rich structure that is typically exploited by humans and machine learning models alike.

#### 6.1.1 Invariances

Imagine that you want to detect an object in an image. It seems reasonable that whatever method we use to recognize objects should not be overly concerned **with the precise *location* of the object**. Ideally, we could learn a system that would somehow exploit this knowledge. Pigs usually don't fly and planes usually do not swim. Nonetheless, we could still recognize a flying pig were one to appear. This idea is taken to the extreme in the children's game "Where's Waldo". The game consists of a number of chaotic scenes bursting with activity and Waldo shows up somewhere in each (typically lurking in some unlikely location). The reader's goal is to locate him. Despite his characteristic outfit, this can be surprisingly difficult, due to the large number of **confounders** 

Back to images, the intuitions we have been discussing could be made more concrete yielding a few key principles for building neural networks for computer vision:

- 1) Our vision systems should, in some sense, respond similarly to the same object **regardless of where it appears in the image** (*translation invariance*)
- 2) Our vision system should, in some sense, focus on **local regions, without regard for what else is happening in the image at greater distances** (*locality*)


#### 6.1.2 Constraining the MLP

To start off, let's consider what an MLP would look like with $h \times w$ images as inputs (represented as matrices in math, and as 2D arrays in code), and hidden representations similarly organized as $h \times w$ / 2D arrays. Let $x[i, j]$ and $h[i, j]$ denote pixel location $(i, j)$ in an image and hidden representation, respectively. Consequently, to have each of the $hw$ hidden nodes receive input from each of the $hw$ inputs, we would switch from using weight matrices (as we did previously in MLPs) to representing our parameters as four-dimensional weight tensors.

We could formally express this dense layer as follows:

$h[i, j] = u[i, j] + \sum{W[i,j,k,l]} * x[k, l] = u[i, j] + \sum{V[i,j,a,b]} * x[i+a, j + b]$

**The switch from W to V is entirely cosmetic (for now) since there is a one-to-one correspondence between coefficients in both tensors**. We simply re-index the subscripts $(k, l)$ such that $k = i + a$ and $l = j + b$. In other words, we set $V[i,j,a,b] = W[i, j, i + a, j + b]$. **The indices $a,b$ run over both positive and negative offsets, covering the entire image. For any given location $(i, j)$ in the hidden layer $h[i, j]$, we compute its value by summing over pixels in $x$, centered around $(i, j)$ and weighted $V[i,j,a,b]$**.

Now, let's invoke the first principle we established above: *translation invariance*. This implies that **a shift in the inputs $x$ should simply lead to a shift in activations $h$**. This is only possible if $V$ and $u$ **DO NOT** actually depend on $(i, j)$, i.e., we have $V[i,j,a,b] = V[a,b]$ and $u$ is a constant. As a result we can simplify the definition for $h$.

$h[i,j] = u + \sum{V[a,b]} * x[i + a, j + b]$

This is a convolution! **We are effectively weighting pixels $(i + a, j + b)$ in the VICINITY of $(i, j)$ with coefficients $V[a,b]$ to obtain the value $h[i,j]$. Note that V[i,j] needs many fewer coefficients than $V[i,j,a,b]$. For a 1 megapixel image it has at most 1 million coefficients**. This is 1 million fewer parameters since it no longer depends on the location within the image. We have made significant progress!

#### 6.1.3 Convolutions

Let's briefly review why the above operation is called a *convolution*. In mathematics, the convolution between two functions, say $f, g: \mathbf{R}^d \rightarrow R$ is defined as:

$[f\circledast{g}](x) = \int_{R^d}f(z)g(x-z)dz$

That is, we measure the overlap between f and g when **both functions are shifted by $x$ and "flipped**. Whenever we have discrete objects, the integral turns into a sum. For instance, for vectors defined on $l_2$, i.e., the set of square summable infinite dimensional vectors with index running over $Z$ we obtain the following definition.

$[f\circledast{g}](i) = \sum_{a}f(a)g(i-a)$


For two-dimensional arrays, we have a corresponding sum with indices $(i, j)$ for $f$ and $(i - a, j - b)$ for $g$ respectively. This looks similar to the definition above, with one major difference. Rather than using $(i + a, j + b)$, we are using the difference instead. Note, though, that this distinction is mostly cosmetic since we can always match the notation by using $V[a,b] = V[-a, -b]$ to obtain $h = x \circledast V$. Also note that the original definition is actually a *cross correlation*. We will come back to this in the following section.

#### 6.1.4 Waldo Revisited

Let's see what this looks like if we want to build an improved Waldo detector. The convolutional layer picks windows of a given size and weighs intensities **according to the mask V**. **We expect that wherever the "waldoness" is highest, we will also find a peak in the hidden layer activations**

There is just a problem with this approach: so far we blissfully ignored that images consist of 3 channels: red, green, and blue. In reality, images aren't quite two-dimensional objects but **rather a $3^{rd}$ order tensor**, e.g., with shape $1024 \times 1024 \times 3$ pixels. Only two of these axes concern spatial relationships, while the $3^{rd}$ can be regarded as assigning a multidimensional representation to *each pixel location*.

We thus index $\textbf{x}$ as $x[i, j, k]$. The convolutional mask has to adapt accordingly. Instead of $V[a, b]$ we now have $V[a, b, c]$

Moreover, just as our input consists of a $3^{rd}$ order tensor, it turns out to be a good idea to similarly formulate our hidden representations as $3^{rd}$ order tensors. In other words, rather than just having a 1D representation corresponding to each spatial location, we want to have a multidimensional hidden representations corresponding to each spatial location. We could think of the hidden representation as comprising a number of 2D grids stacked on top of each other. These are sometimes called *channels* or *feature maps*. Intuitively you might imagine that at lower layers, some channels specialize at recognizing edges for example. We can take care of this by adding a fourth coordinate to $V$ via $V[a, b, c, d]$.


#### Summary

- Translation invariance in images implies that all patches of an image will be treated in the same manner

- Locality means that only a small neighborhood of pixels will be used for computation

- Channels on input and output allow for meaningful feature analysis.

### 6.2 Convolutions for Images

#### 6.2.1 The Cross-Correlation Operator

In a convolutional layer, an **input array** and a **correlation kernel array** are combined to produce an output array through a cross-correlation operation. Let's see how this works for two dimensions. The input in our example is a 2D array with a height of 3 and width of 3. We mark the shape of the array as $3 \times 3$ or (3,3). The height and width of the kernel array are both 2. Common names for this array in the deep learning community include *kernel* and *filter*. The shape of the kernel window (also known as the convolutional window) is given precisely by the height and width of the kernel (here it is $2 \times 2$)

In [3]:
from mxnet import autograd, np, npx
from mxnet.gluon import nn

npx.set_np()

In [4]:
def corr2d(X, K):
    '''Compute 2D cross-correlation.'''
    h, w = K.shape
    Y = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [5]:
X = np.array([[0,1,2], 
              [3,4,5], 
              [6,7,8]])
K = np.array([[0, 1], 
              [2, 3]])

In [6]:
corr2d(X, K)

array([[19., 25.],
       [37., 43.]])

In [7]:
K.shape

(2, 2)

#### 6.2.2 Convolutional Layers

A convolutional layer **cross-correlates** the input and kernels and adds a scalar bias to produce an output. The parameters of the convolutional layer are precisely the values that constitute the kernel and the scalar bias. When training the models based on convolutional layers, we typically initialize the kernels randomly, just as we would with a fully-connected layer.

We are now ready to implement a 2D-convolutional layer based on the corr2d function defined above. In the `__init__` constructor function, we declare weight and bias as the two model parameters. The forward computation function *forward* calls the corr2d function and adds the bias. As with $h \times w$ cross-correlation we also refer to convolutional layers as $h \times w$ convolutions.

In [8]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape = kernel_size)
        self.bias = self.params.get('bias', shape = (1,))
        
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

#### 6.2.3 Object Edge Detection in Images

Let's look at a simple application of a convolutional layer: detecting the edge of an object in an image **by finding the location of the pixel change**. First, we construct an 'image' of $6 \times 8$ pixels. The middle four columns are black (0) and the rest are white (1).

In [13]:
X = np.ones((6,8))

In [14]:
X[:, 2:6] = 0
X

array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]])

Next, we construct a kernel K with a height of 1 and width of 2. When we perform the cross-correlation operation with the input, ***if the horizontally adjacent elements are the same, the output is zero***. Otherwise, the output is non-zero.

In [19]:
K = np.array([[1, -1]])
K_2 = np.array([[1],
                [-1]])

Enter X and our designed kernel K to perform the cross-correlation operations. As you can see, we will detect 1 for the edge from white to black and -1 for the edge from black to white. The rest of the outputs are 0. 

In [16]:
Y = corr2d(X, K)
Y

array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

Let's apply the kernel to the transposed image. As expected, it vanishes. **The kernel K only detects vertical edges.**

Note: What would happen if I were to transpose the kernel so that it is represented as a column vector? (2 x 1 in this case)
Answer: It would work correctly, i.e. the result would be the transpose of Y (that is calculated above)

In [22]:
corr2d(X.T, K_2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1., -1., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

#### 6.2.4 Learning a Kernel

Designing an edge detector by finite differences [1, -1] is neat if we know this is precisely what we are looking for. However, as we look at larger kernels, and consider successive layers of convolutions, it might be impossible to specify precisely what each filter should be doing manually. 

Now, let's see whether we can **learn the kernel** that generated Y from X by looking at the (input, output) pairs only. We first construct a convolutional layer and initialize its kernel as a random array. Next, in each iteration, we will use the **squared error* to compare Y and the output of the convolutional layer, then calculate the gradient to update the weight**. For the sake of simplicity, in this convolutional layer, we will ignore the bias.

We previously constructed the Conv2D class. However, since we used single-element assignments, Gluon has some trouble finding the gradient. Instead, we use the built-in Conv2D class provided by Gluon.

In [24]:
# Construct a convolutional layer with 1 output channel
# (channels will be introduced in the following sections)
# and a kernel array shape of (1,2)

conv2d = nn.Conv2D(1, kernel_size=(1, 2))

conv2d.initialize()

The two dimensional convolutional layer **uses four-dimensional input** and output in the format of: 

- (example channel, height, width)

where the batch size and the number of channels are both 1.

In [25]:
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

In [26]:
for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    # Ignoring bias, for the sake of simplicity
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print(f'batch:{i + 1}, loss: {l.sum()}')

batch:2, loss: 4.9493217
batch:4, loss: 0.83127165
batch:6, loss: 0.13983817
batch:8, loss: 0.02361415
batch:10, loss: 0.0040245447


As you can see, the error has dropped to a small value after 10 iterations. Now, we will take a look at the kernel array we learned.

In [29]:
conv2d.weight.data().reshape(1, 2)

array([[ 0.9895   , -0.9873705]])

#### 6.2.5 Cross-Correlation and Convolution

Recall the observation from the previous section that **cross-correlation and convolution are equivalent**. In the figure above it is easy to see this correspondence. Simply flip the kernel from the bottom left to the top right. In this case the indexing in the sum is reverted, yet the same result can be obtained. In keeping with standard terminology with deep learning literature, we will continue to refer to the cross-correlation operation as a convolution even though, strictly speaking, it is slightly different.

#### Summary

- The core computation of a two-dimensional convolutional layer is a two-dimensional cross correlation operation. In its simplest form, this performs a cross-correlation on the two-dimensional input data and the kernel, and then adds a bias.

- We can design a kernel to detect edges in images

- We can learn the kernel through data.

### 6.3 Padding and Stride